In [2]:
import pandas as pd

import json

In [3]:
df_full = pd.read_csv("data/init_data.csv")
df_full.head(2)

C:\Users\glaz\AppData\Local\Temp\ipykernel_20440\263171808.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("data/init_data.csv")


,addOkved,address,chiefs,dateLiquid,dateOgrn,dateReg,email,fullName,inn,kpp,...,okato,okfs,okogu,oktmo,socialMedia,opfType,reports,incomeTotal,incomeDetail,incomeRank
0,"[{'code': '72.4', 'name': 'Деятельность по соз...","422430, РЕСПУБЛИКА ТАТАРСТАН, ГОРОД БУИНСК, УЛ...","[{'name': 'ХАЛИКОВ РАШИТ РИНАТОВИЧ', 'title': ...",{'$date': '2012-12-17T00:00:00.000Z'},{'$date': '2010-03-03T00:00:00.000Z'},NaN,NaN,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""ИНФОРМАЦИОННО-КОНС...",1.614010e+09,161401001.0,...,"{'code': None, 'name': None}","{'code': None, 'name': None}","{'code': None, 'name': None}","{'code': None, 'name': None}","{'facebook': None, 'instagram': None, 'ok': No...",Некоммерческие партнерства,[],0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",Наличие доходов из доступных источников не уст...
1,"[{'code': '67.20.2', 'name': 'Деятельность спе...","430030, РЕСПУБЛИКА МОРДОВИЯ, ГОРОД САРАНСК, УЛ...","[{'name': 'ПУЗАНОВ ВЛАДИМИР ВЛАДИМИРОВИЧ', 'ti...",{'$date': '2014-11-10T00:00:00.000Z'},{'$date': '2010-05-11T00:00:00.000Z'},NaN,NaN,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""МЕЖРЕГИОНАЛЬНЫЙ НА...",1.327011e+09,132701001.0,...,"{'code': None, 'name': None}","{'code': None, 'name': None}","{'code': None, 'name': None}","{'code': None, 'name': None}","{'facebook': None, 'instagram': None, 'ok': No...",Некоммерческие партнерства,"[{'type': 'Заявление', 'reportId': '19445801',...",0.0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",Наличие доходов из доступных источников не уст...


Формируем признаки:
1. totalSumGrants (тип floats) - сумма грантов и субсидий
2. totalSumContract (тип floats) - сумма контрактов

Исходные данные для формирования признаков - признак incomeDetail. 

In [4]:
# Функция для вычисления суммы totalSumGrants для "grants" и "fedSubsidies"
def calculate_total_sum_grant(json_string):
    #Аргумент "json_string" - строка с данными о сумме и количеству грантов, субсидий и контрактов
    try:
        # Преобразование JSON-строки        
        income_detail_dict = eval(json_string)
        # Получение значений totalSum для "grants" и "fedSubsidies"
        grants_total_sum = income_detail_dict.get("grants", {}).get("totalSum", 0)
        fed_subsidies_total_sum = income_detail_dict.get("fedSubsidies", {}).get("totalSum", 0) 
        # Возвращение суммы totalSumGrants типа float
        return float(grants_total_sum) + float(fed_subsidies_total_sum)
    except:
        return 0

# Функция для вычисления суммы totalSum для "grants" и "fedSubsidies"
def calculate_total_sum_contract(json_string):
    #Аргумент "json_string" - строка с данными о сумме и количеству грантов, субсидий и контрактов
    try:
        # Преобразование JSON-строки        
        income_detail_dict = eval(json_string)
        # Получение значений totalSum для контрактов: contracts44, contracts94, contracts223
        total_sum_44 = income_detail_dict.get("contracts44", {}).get("totalSum", 0)
        total_sum_94 = income_detail_dict.get("contracts94", {}).get("totalSum", 0)
        total_sum_223 = income_detail_dict.get("contracts223", {}).get("totalSum", 0)
        # Возвращение суммы totalSumContract типа float
        return float(total_sum_44) + float(total_sum_94) + float(total_sum_223)
    except:
        return 0

# Формирование признаков totalSumGrants и totalSumContract
df_full['totalSumGrants'] = df_full['incomeDetail'].apply(calculate_total_sum_grant)
df_full['totalSumContract'] = df_full['incomeDetail'].apply(calculate_total_sum_contract)

In [5]:
df_full[df_full['incomeTotal'] > 0][['incomeTotal','totalSumGrants','totalSumContract']]

,incomeTotal,totalSumGrants,totalSumContract
7,14770110.00,0.0,14770110.00
11,15000.00,0.0,15000.00
91,270436.00,270436.0,0.00
100,164450.00,0.0,164450.00
105,159800.00,0.0,159800.00
...,...,...,...
751965,82932.08,0.0,82932.08
751970,55296779.68,0.0,55296779.68
751990,4910000.00,4910000.0,0.00
751995,8288.42,0.0,8288.42
